<a href="https://colab.research.google.com/github/Febilid/Pem_Mesin/blob/main/Week_10_Praktikum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Sheet 10: Recurrent Neural Network (RNN)
# Nama: Febiola lidya Sianturi
# NIM: 2241720229
# Kelas: TI-3H
# No: 10

# Praktikum 1 RNN untuk Analisis Sentimen
# Setup
Impor matplotlib dan buat fungsi pembantu untuk memplot grafik:

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [2]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

# Setup input pipeline

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'Conrad Hall went out with a bang. The great film photographer finished his illustrious career with this movie before passing on. He did himself proud as this is one of the best-looking crime films you\'ll ever see.<br /><br />Of course, the acting ain\'t bad when you have Tom Hanks and Paul Newman playing the leads! The amount of action in here is just right, too: not too much; not too little.<br /><br />None of the characters in here, frankly, are "good guys" as Hanks is a professional hit-man for town boss Newman. Hanks\' only redeeming quality is not wanting his young son to wind up a killer like him, although he does teach him how to be the getaway man in robberies! Huh?<br /><br />As good as the acting is and as interesting as the story is, the real star of this film is cinematographer Hall, who paints scene after beautiful scene with his lens. His work is just awesome.'
 b'Reasons to watch the movie:<br /><br />1) Bo Derek at 16 looks good and occasionally gets naked. 

# Buat Teks Encoder

In [6]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [7]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [8]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1,   1, 418, ...,   0,   0,   0],
       [  1,   6, 104, ...,   0,   0,   0],
       [ 30,   1,   1, ...,   0,   0,   0]])

In [9]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'Conrad Hall went out with a bang. The great film photographer finished his illustrious career with this movie before passing on. He did himself proud as this is one of the best-looking crime films you\'ll ever see.<br /><br />Of course, the acting ain\'t bad when you have Tom Hanks and Paul Newman playing the leads! The amount of action in here is just right, too: not too much; not too little.<br /><br />None of the characters in here, frankly, are "good guys" as Hanks is a professional hit-man for town boss Newman. Hanks\' only redeeming quality is not wanting his young son to wind up a killer like him, although he does teach him how to be the getaway man in robberies! Huh?<br /><br />As good as the acting is and as interesting as the story is, the real star of this film is cinematographer Hall, who paints scene after beautiful scene with his lens. His work is just awesome.'
Round-trip:  [UNK] [UNK] went out with a [UNK] the great film [UNK] [UNK] his [UNK] career with th

# Buat Model

In [10]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [11]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [12]:
import numpy as np
import tensorflow as tf

# Definisikan teks sampel
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')

# Tokenisasi teks menggunakan tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([sample_text])
tokenized_sequence = tokenizer.texts_to

AttributeError: 'Tokenizer' object has no attribute 'texts_to'